# K26 - Heated Wall

Interface at 90°.  
Variable fluid densities => but very small variations  
Also no Heat capacity => infinitely fast heat conduction  
Height of the domain is reduced  
Implicit RK timestepping, to allow for an adaption of the maximum step size  
Also only the "start" of the process is simulated, i.e. tE = 1

In [ ]:
#r "..\..\..\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Setup Workflowmanagement, Batchprocessor and Database

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths,BatchInstructionDir
0,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,<null>,dotnet,FDY\rieckmann,DC2,<null>,Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann, PathAtRemote: } ]",
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER,dotnet,FDY\rieckmann,DC2,[ hpccluster ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER2,dotnet,FDY\rieckmann,DC2,[ hpccluster2 ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
3,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\rieckmann\AppData\Local\BoSSS-LocalJobs,False,<null>,dotnet,,,,,,,<null>


In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
static var myDb = myBatch.CreateOrOpenCompatibleDatabase("XNSFE_HeatedWall");

In [ ]:
myDb.Path

Opening existing database '\\hpccluster\hpccluster-scratch\rieckmann\XNSFE_HeatedWall'.


\\hpccluster\hpccluster-scratch\rieckmann\XNSFE_HeatedWall

In [ ]:
BoSSSshell.WorkflowMgm.Init($"HeatedWall_VariableDensity");

Project name is set to 'HeatedWall_VariableDensity'.


## Setup Simulationcontrols

In [ ]:
using BoSSS.Application.XNSFE_Solver;

In [ ]:
int[] hRes = {16, 24, 32, 48};
int[] pDeg = {2};
double[] Q = {0.2};
double[] dRho_B = {0.0, -1e-3, -5e-3, -1e-2, -5e-2, -1e-1};

In [ ]:
List<XNSFE_Control> Controls = new List<XNSFE_Control>();
foreach(int h in hRes){
    foreach(int p in pDeg){
        foreach(double q in Q){
            foreach(double dR in dRho_B){

                var ctrl = new XNSFE_Control();
                ctrl.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("HeatFlux", q));

                ctrl.DbPath      = null;
                ctrl.SessionName = $"HeatedWall_VariableDensity_res:{h}_p:{p}_dR:{dR}";
                ctrl.ProjectName = $"HeatedWall_VariableDensity";
                ctrl.SetDatabase(myDb);
                ctrl.savetodb = true;        

                ctrl.FieldOptions.Add("VelocityX", new FieldOpts() {
                    Degree = p,
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("VelocityY", new FieldOpts() {
                    Degree = p,
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("GravityX#A", new FieldOpts() {
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("GravityY#A", new FieldOpts() {
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("GravityX#B", new FieldOpts() {
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("GravityY#B", new FieldOpts() {
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("Pressure", new FieldOpts() {
                    Degree = p - 1,
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("PhiDG", new FieldOpts() {
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("Phi", new FieldOpts() {
                    Degree = Math.Max(p, 2),
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });
                ctrl.FieldOptions.Add("Temperature", new FieldOpts() {
                    Degree = p,
                    SaveToDB = FieldOpts.SaveToDBOpt.TRUE
                });

                #region grid
                double L = 5.0;
                int kelemR = h;
                string[] Bndy = new string[] {  "Inner",
                                                "NavierSlip_linear_ConstantHeatFlux_right",
                                                "pressure_outlet_ZeroGradient_top",
                                                "freeslip_ZeroGradient_left",
                                                "pressure_outlet_ZeroGradient_bottom"};

                ctrl.GridFunc = delegate () {
                    double[] Xnodes = GenericBlas.Linspace(-L, 0, kelemR + 1);
                    double[] Ynodes = GenericBlas.Linspace(0, L, kelemR + 1);
                    var grd = Grid2D.Cartesian2DGrid(Xnodes, Ynodes);

                    for(byte i= 1; i < Bndy.Count(); i++) {
                        grd.EdgeTagNames.Add(i, Bndy[i]);
                    }

                    grd.DefineEdgeTags(delegate (double[] X) {
                        byte et = 0;
                        if(Math.Abs(X[0] - Xnodes.Last()) < 1e-8)
                            return 1;
                        if(Math.Abs(X[0] - Xnodes.First()) < 1e-8)
                            return 3;
                        if(Math.Abs(X[1] - Ynodes.Last()) < 1e-8)
                            return 2;
                        if(Math.Abs(X[1] - Ynodes.First()) < 1e-8)
                            return 4;
                        return et;
                    });

                    return grd;
                };
                #endregion

                #region material
                ctrl.PhysicalParameters = new BoSSS.Solution.XNSECommon.PhysicalParameters() {
                    rho_A = 1.0, // 958.0
                    rho_B = 1.0 + dR, // 0.59,

                    mu_A = 1, //2.82 * 1e-4,
                    mu_B = 0.001, //1.23 * 1e-6,

                    Sigma = 1.0,
                    betaS_A = 1000, // sliplength is mu/beta
                    betaS_B = 1000,
                };

                ctrl.ThermalParameters = new BoSSS.Solution.XheatCommon.ThermalParameters() {
                    rho_A = 1.0, // 958.0
                    rho_B = 1.0 + dR, //0.59,

                    k_A = 1.0, // 0.6
                    k_B = 1.0, // 0.026,

                    c_A = 0.0,
                    c_B = 0.0,

                    hVap = 1,//2.257 * 1e6,
                    T_sat = 0.0 // 373.0
                };

                ctrl.PhysicalParameters.IncludeConvection = true;
                ctrl.ThermalParameters.IncludeConvection = true;
                ctrl.PhysicalParameters.Material = false;
                #endregion

                #region Initial Condition - Exact Solution

                // solution for massflux and velocity at level set
                double y0 = 0.2 * L;

                // inital values
                double g = 4;
                ctrl.AddInitialValue("Phi", $"(X, t) => -{y0} + X[1]", true);
                ctrl.AddInitialValue("Temperature#A", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
                ctrl.AddInitialValue("Temperature#B", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
                ctrl.AddInitialValue("GravityY#A", $"(X, t) => -{g}", true);

                #endregion

                #region Boundary Conditions

                double v = 1.0;
                ctrl.AddBoundaryValue(Bndy[1], "HeatFluxX#A", $"(X, t) => {q}", true);
                ctrl.AddBoundaryValue(Bndy[1], "VelocityY#A", $"(X, t) => {v}", true);
                ctrl.AddBoundaryValue(Bndy[1], "VelocityY#B", $"(X, t) => {v}", true);


                ctrl.AddBoundaryValue(Bndy[3]);
                ctrl.AddBoundaryValue(Bndy[2]);
                ctrl.AddBoundaryValue(Bndy[4], "Pressure#A", $"(X, t) => {y0} * {ctrl.PhysicalParameters.rho_A} * {g}", true);

                #endregion

                #region AMR

                // No AMR
                int level = 0;
                ctrl.AdaptiveMeshRefinement = level > 0;
                ctrl.activeAMRlevelIndicators.Add(new BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater.AMRonNarrowband() { maxRefinementLevel = level });
                ctrl.AMR_startUpSweeps = level;

                #endregion

                #region Timestepping

                ctrl.AdvancedDiscretizationOptions.SST_isotropicMode = BoSSS.Solution.XNSECommon.SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
                ctrl.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.FastMarching;
                ctrl.Timestepper_LevelSetHandling =  BoSSS.Solution.XdgTimestepping.LevelSetHandling.LieSplitting;

                ctrl.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
                ctrl.NonLinearSolver.Globalization = BoSSS.Solution.AdvancedSolvers.Newton.GlobalizationOption.Dogleg;
                ctrl.NonLinearSolver.ConvergenceCriterion = 1e-8;
                ctrl.NonLinearSolver.MaxSolverIterations = 10;

                ctrl.SkipSolveAndEvaluateResidual = false;

                ctrl.TimeSteppingScheme = BoSSS.Solution.XdgTimestepping.TimeSteppingScheme.RK_ImplicitEuler;
                ctrl.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Transient;
                ctrl.dtFixed = 0.01;
                ctrl.Endtime = 1.0;
                ctrl.NoOfTimesteps = int.MaxValue; // timesteps can be adapted, simulate until endtime is reached

                #endregion
                ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MassfluxLogging() { LogPeriod = 1 });
                ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MovingContactLineLogging() { LogPeriod = 1 });

                Controls.Add(ctrl);
            }
        }
    }
}

In [ ]:
Controls.Count

24

## Start simulations on Batch processor

In [ ]:
foreach(var C in Controls) {
    Type solver = typeof(BoSSS.Application.XNSFE_Solver.XNSFE<XNSFE_Control>);

    string jobName       = C.SessionName;
    var oneJob           = new Job(jobName, solver);
    oneJob.NumberOfMPIProcs = 1;    
    oneJob.SetControlObject(C);
    oneJob.Activate(myBatch, true);
}